In [27]:
#importing modules 
import json 
import pandas as pd
import numpy as np
import csv
from fuzzywuzzy import fuzz, process
import re
from timeit import default_timer as timer

# test file path
path = "./test_directory_files/"
tsv = "tsv/"
locations = "4b119360-317a-0134-9131-00505686a51c_locations.tsv"
occupations = "4b119360-317a-0134-9131-00505686a51c_occupations.tsv"
subjects = "4b119360-317a-0134-9131-00505686a51c_subjects.tsv"
test_json = "119.56837603.5abc93e0-6e04-0134-173e-00505686a51c_labeled.json"
street_names_txt = "./city-directory-entry-parser-master/streetnames.txt"

In [28]:
# Open tsvs and streetnames
## From Nick: Columns are the directory uuid, page uuid, entry uuid, the number (in 0,1,2,3 index order) of the entry 
##            (remember that there could be more than one subject per entry, more than one occupation, etc.).Most are 
##            entry 0, but sometimes you might find an entry 1 if there are multiple subjects/occupations/locations. 
##            Then an offset number, e.g. the order of the token from the start of the entry. So a 0th token, 1st token,
##            2nd token, etc. And finally the token itself in the last column.
##            Remember that we have to keep that entry and offset number linked to each token otherwise we won't be able 
##            to swap any corrections back into the JSON file once we have a listed of recommended edits.

locations_tsv = pd.read_csv(path + tsv + locations, 
                           sep = "\t",
                           quoting = csv.QUOTE_NONE)
occupations_tsv = pd.read_csv(path + tsv + occupations, 
                           sep = "\t",
                           quoting = csv.QUOTE_NONE)
subjects_tsv = pd.read_csv(path + tsv + subjects, 
                           sep = "\t",
                           quoting = csv.QUOTE_NONE)
master_street_names = pd.read_csv(street_names_txt, 
                                  header = None)
master_street_names = master_street_names[0].tolist()
#master_street_names

In [29]:
print(locations_tsv.entry_uuid.nunique())
#locations_tsv.head(n = 20)

164615


In [30]:
# function remove punctuation and default to lowercase
def clean_token(token):
    token_lower = token.lower()
    token_clean = ''.join(t for t in token_lower if t.isalnum())
    return token_clean

# apply to column and create new column
locations_tsv["token_re"] = locations_tsv["token"].apply(clean_token)

# drop rows with just non alpha-numeric characters
locations_tsv = locations_tsv.drop(locations_tsv[(locations_tsv.token_re == '')].index)

# create trial pd
locations_tsv_trunc = locations_tsv.sample(n = 100)
#locations_tsv_trunc.head(n=50)

In [33]:
#start = timer()

def fuzzy_match(x, choices, scorer):
    # check if numeric:
    digit_check = x.isdigit()
    if digit_check is True:
        #print("numeric")
        return x
    else:
        #print("non-numeric")
        # best_guess = process.extractOne(x, choices=choices, scorer=scorer)
        ## use the first return if you want to see similarity score, otherwise, use the second
        return process.extractOne(x, choices=choices, scorer=scorer)        
        #return process.extractOne(x, choices=choices, scorer=scorer)[0]

# create a second truncated df to trial
locations_tsv_trunc_2 = locations_tsv_trunc
    
# Following post explains why token_sort_ratio might be best 
## https://medium.com/analytics-vidhya/matching-messy-pandas-columns-with-fuzzywuzzy-4adda6c7994f
## tldr: converts each word to a token, then sorts aphabetically. 
## Streetnames that are abbrievated and in full both start with the same letter so this avoids errors like bway -> w  
locations_tsv_trunc_2["token_matched"] = locations_tsv_trunc_2["token_re"].apply(
    fuzzy_match,
    args=(
        master_street_names,
        fuzz.token_sort_ratio
    )
)

#end = timer()
#print(end - start)

In [34]:
locations_tsv_trunc_2

# Issues and errors examples:
## False negatives: bway -> broadway, 67% similar 
## False positives: 127th -> 12th, 89% similar
## True positives: grand -> Grand, 100% similar
## True negatives: 68d -> dry, 33% similar | pl -> Pell, 67% similar

,directory_uuid,page_uuid,entry_uuid,location_count,offset_count,token,token_re,token_matched
231009,4b119360-317a-0134-9131-00505686a51c,400.56767356.497fc4f0-5361-0134-c8c4-00505686a51c,f391c8ec499e11ea8ccce0accb5fbfb2,0,0,av.,av,"(Av, 100)"
356654,4b119360-317a-0134-9131-00505686a51c,651.56767607.7f6097a0-5361-0134-beb4-00505686a51c,ccec672c49a011eaa112e0accb5fbfb2,0,0,Av.,av,"(Av, 100)"
102313,4b119360-317a-0134-9131-00505686a51c,36.56766992.28a347b0-5361-0134-43e2-00505686a51c,9ee3fea648e411eaae85e0accb5fbfb2,0,0,Wall,wall,"(Wall, 100)"
291231,4b119360-317a-0134-9131-00505686a51c,519.56767475.738c4430-5361-0134-275a-00505686a51c,d04f0466499f11ea830de0accb5fbfb2,0,0,av.,av,"(Av, 100)"
348728,4b119360-317a-0134-9131-00505686a51c,635.56767591.7dd94bf0-5361-0134-493b-00505686a51c,adf9131a49a011eaacece0accb5fbfb2,0,0,EB.,eb,"(E, 67)"
...,...,...,...,...,...,...,...,...
213842,4b119360-317a-0134-9131-00505686a51c,366.56767322.466394b0-5361-0134-2932-00505686a51c,b694da7e499e11ea987ce0accb5fbfb2,0,0,Fifth,fifth,"(Fifth, 100)"
124911,4b119360-317a-0134-9131-00505686a51c,80.56767036.2c80f450-5361-0134-74cd-00505686a51c,417f5bb448e611ea84c5e0accb5fbfb2,0,0,96,96,96
457927,4b119360-317a-0134-9131-00505686a51c,848.56767804.91966ba0-5361-0134-6545-00505686a51c,4b04600a49a211ea8042e0accb5fbfb2,0,0,Waverley,waverley,"(Waverley, 100)"
338350,4b119360-317a-0134-9131-00505686a51c,614.56767570.7c0f3100-5361-0134-a006-00505686a51c,8646a51c49a011ea973ee0accb5fbfb2,0,0,Sixth,sixth,"(Sixth, 100)"


In [ ]:
print(occupations_tsv.entry_uuid.nunique())
occupations_tsv.head(n = 20)

In [ ]:
print(subjects_tsv.entry_uuid.nunique())
subjects_tsv.head(n = 20)

## Discarded | Obsolete Code

[Link](https://stackoverflow.com/questions/40712178/reading-the-json-file-with-multiple-objects-in-python) to help with decoding this particular JSON object. Multiple JSON object within one.

In [ ]:
# Opens the file and saves it to a pandas df
##with open(path + test_json) as f:
##    df = pd.read_json(f, lines=True)

In [ ]:
# Quick look into the JSON table to check
##df.tail()

In [ ]:
# Manual uniqueness check
##line_number = np.array(df.original_hocr_line_number)
##unique_elements, counts_elements = np.unique(line_number, return_counts=True)
##print("Frequency of unique values of the said array:")
##print(np.asarray((unique_elements, counts_elements)))

Labels "a", "h", and "r" represent the relative addresses, and address types such as houses, and rear.

In [ ]:
# Quick glimpse of how the last column might actually look like
## df.iloc[124,-1]

In [ ]:
##df.iloc[110,-1]

In [ ]:
##df.iloc[10,-1]

In [ ]:
##df.iloc[4,-1]

In [ ]:
##df.iloc[147, -1]

In [ ]:
##df_labeled_entry = df.labeled_entry.apply(pd.Series)
##df_locations = df_labeled_entry["locations"]
##df_locations = pd.concat([df_locations, df.original_hocr_line_number], axis = 1)

In [ ]:
##df_locations

In [ ]:
##df_locations.iloc[147,:]

In [ ]:
##type(df_locations.iloc[147,0])

In [ ]:
##type(df_locations.iloc[147,0][1])

In [ ]:
##df_locations.iloc[147][0]

In other words, each labeled entry's location is stored as a list of _n_ dictionaries with two key-value pairs of `value` and `labels`.

In [ ]:
##df_locations.iloc[3][0]

In [ ]:
##type(df_locations)

In [ ]:
# This checks for existence of labels, duly records the (non)existence, strips it out, appends to another list
# This is done w/o loss to order

#line_num_list = []
#complete_street_name = []
#split_street_name = []
#labels = []
#for entry in range(0,len(df_locations)):
#    #print(df_locations.iloc[entry])
#    for dicts in df_locations.iloc[entry][0]:
#        if 'labels' in dicts:
#            for key, value in dicts.items():
#                #line_num_list.append(df_locations.iloc[entry][1])
#                if key == 'labels':
#                    labels.append(value)
#                else:
#                    complete_street_name.append(value)
#                    street_name_split = str(value).split()
#                    split_street_name.append(street_name_split)
#                    line_num_list.append(df_locations.iloc[entry][1])
#        else:
#            for key, value in dicts.items():
#                line_num_list.append(df_locations.iloc[entry][1])
#                labels.append(" ")
#                complete_street_name.append(value)
#                street_name_split = str(value).split()
#                split_street_name.append(street_name_split)


It appears that avenues are spelled out and streets are enumerated.

In [ ]:
#split_street_name

In [ ]:
#unlisted_location = pd.concat([pd.Series(complete_street_name), pd.Series(split_street_name), pd.Series(labels), pd.Series(line_num_list)], axis = 1)
#unlisted_location.columns = ["complete_street_name", "split_street_name", "labels", "line_number"]

In [ ]:
#len(line_num_list)

In [ ]:
#unlisted_location

In [ ]:
# first order of business is to test if string is a street number
# if it is not, then it is probably a street name, and a comparison can be made
# Perform regex to remove the periods
# https://www.datacamp.com/community/tutorials/fuzzy-string-python
# https://datascience.stackexchange.com/questions/29775/how-to-check-and-correct-misspelling-in-the-data-of-pairs-of-words
# http://blog.keyrus.co.uk/fuzzy_matching_101_part_i.html
# http://blog.keyrus.co.uk/fuzzy_matching_101_part_ii.html

#for i in range(0, len(unlisted_location)):
#    non_numeric_street_name = []
#    for address_component in unlisted_location.iloc[i, 1]:
#        if address_component.isnumeric() == False:
#            non_numeric_street_name.append(address_component)
#            print(non_numeric_street_name)
#        unlisted_location.iloc[i]["non_numeric"] = non_numeric_street_name
            
#        #print(address_component.isnumeric())


In [ ]:
#unlisted_location